In [4]:
from arcgis.gis import GIS
gis = GIS("home")

In [5]:
import pandas as pd
import arcpy

File we want to push as an update

In [3]:
df = pd.read_csv("C:\ArcGIS\data_upload\historical\merged_output_fixed.csv")
df.head()

,date,TARGET_FID,Unnamed: 0,date_str,humidity,precipitation,solar_rad,temperature_max,temperature_mean,temperature_min,wind
0,2023-01-01,0,20587.0,01/01/2023,0.002384,0.000000,69.957150,25.068811,19.943432,16.217112,1.764542
1,2023-01-02,0,61572.0,01/02/2023,0.001854,0.000000,83.297345,24.800823,16.414769,9.901263,2.329689
2,2023-01-03,0,103557.0,01/03/2023,0.001596,0.000000,80.083720,22.508005,16.494677,11.931353,4.046819
3,2023-01-04,0,145042.0,01/04/2023,0.001889,0.000000,82.971116,25.257251,17.898934,14.511572,4.351736
4,2023-01-05,0,187027.0,01/05/2023,0.002305,0.002071,65.155722,30.377338,23.178682,18.850564,6.389858


File we want to match the column names to

In [4]:
df2 = pd.read_csv("C:\ArcGIS\data_upload\historical\merged_output.csv")
df2.head()

,date,TARGET_FID,humidity,precipitation,solar_rad,temperature_max,temperature_mean,temperature_min,wind
0,2025-07-21,0,0.006933,0.0,284.780109,73.322397,62.037954,52.594761,2.867311
1,2025-07-21,1,0.006884,0.0,285.547820,72.086089,59.876542,50.402426,2.939255
2,2025-07-21,2,0.006666,0.0,286.527856,76.347150,64.264003,53.768075,2.754215
3,2025-07-21,3,0.006965,0.0,283.599931,70.846138,59.593824,50.393193,2.790148
4,2025-07-21,4,0.006839,0.0,284.791686,70.502457,58.211662,49.090770,2.831708


In [5]:
# Drop unnecessary columns
df = df.drop(columns=['Unnamed: 0', 'date_str'], errors='ignore')

# Reorder columns to match df2
df = df[df2.columns]
df.head()

,date,TARGET_FID,humidity,precipitation,solar_rad,temperature_max,temperature_mean,temperature_min,wind
0,2023-01-01,0,0.002384,0.000000,69.957150,25.068811,19.943432,16.217112,1.764542
1,2023-01-02,0,0.001854,0.000000,83.297345,24.800823,16.414769,9.901263,2.329689
2,2023-01-03,0,0.001596,0.000000,80.083720,22.508005,16.494677,11.931353,4.046819
3,2023-01-04,0,0.001889,0.000000,82.971116,25.257251,17.898934,14.511572,4.351736
4,2023-01-05,0,0.002305,0.002071,65.155722,30.377338,23.178682,18.850564,6.389858


Reform the date column to account for timezones

In [6]:
df['date'] = pd.to_datetime(df['date'], errors='coerce')
df['date'] = df['date'].apply(lambda dt: dt.replace(hour=8) if pd.notnull(dt) else dt)

In [7]:
df.head()

,date,TARGET_FID,humidity,precipitation,solar_rad,temperature_max,temperature_mean,temperature_min,wind
0,2023-01-01 08:00:00,0,0.002384,0.000000,69.957150,25.068811,19.943432,16.217112,1.764542
1,2023-01-02 08:00:00,0,0.001854,0.000000,83.297345,24.800823,16.414769,9.901263,2.329689
2,2023-01-03 08:00:00,0,0.001596,0.000000,80.083720,22.508005,16.494677,11.931353,4.046819
3,2023-01-04 08:00:00,0,0.001889,0.000000,82.971116,25.257251,17.898934,14.511572,4.351736
4,2023-01-05 08:00:00,0,0.002305,0.002071,65.155722,30.377338,23.178682,18.850564,6.389858


Save to CSV

In [8]:
df.to_csv("C:\ArcGIS\data_upload\historical\cleaned_for_gdb_2.csv")

Turn into a geodatabase

In [9]:
import os
# Define paths
out_folder = r"C:\ArcGIS\data_upload\historical"
gdb_name = "upload_ready_07_28_2025_datetime.gdb"
gdb_path = os.path.join(out_folder, gdb_name)

# Create File GDB if it doesn't exist
if not arcpy.Exists(gdb_path):
    arcpy.CreateFileGDB_management(out_folder, gdb_name)

# Convert cleaned CSV to GDB Table
input_csv = os.path.abspath("C:\ArcGIS\data_upload\historical\cleaned_for_gdb_2.csv")  # Full path
arcpy.TableToTable_conversion(
    input_csv,
    gdb_path,
    "merged_output"  # This name must match what ArcGIS expects to append to
)

<Result 'C:\\ArcGIS\\data_upload\\historical\\upload_ready_07_28_2025_datetime.gdb\\merged_output'>

Zip geodatabase

In [11]:
import shutil
# Path to the zip file (must end in .zip)
gdb_zip_path = "C:\ArcGIS\data_upload\historical\output_07_28_2025.zip"

# Zip the .gdb folder (make_archive auto adds .zip unless you give full path)
shutil.make_archive(base_name=gdb_zip_path.replace('.zip',''), format='zip', root_dir=os.path.dirname(gdb_path), base_dir=os.path.basename(gdb_path))

'C:\\ArcGIS\\data_upload\\historical\\output_07_28_2025.zip'

The next two code cells will delete data based on date. Useful for if you accidently upload 32 million points incorrectly

In [6]:
# Get the existing hosted feature layer (the target layer you want to update)
target_item = gis.content.get("e7c7151adfd549239c31f9cc2363831c")
layer = target_item.tables[0]

In [7]:
from datetime import datetime, timedelta
import time

# Define the date range
start_date = datetime(2023, 5, 30)
end_date = datetime(2025, 4, 19)

# Loop through each day
current_date = start_date
total_deleted = 0

while current_date <= end_date:
    date_str = current_date.strftime('%Y-%m-%d')
    
    # Full-day where clause
    where_clause = (
        f"date BETWEEN TIMESTAMP '{date_str} 00:00:00' AND TIMESTAMP '{date_str} 23:59:59'"
    )

    try:
        # Try to delete that day's records
        response = layer.delete_features(where=where_clause)
        deleted = response.get("deleteResults", [])
        if deleted:
            print(f"✅ Deleted {len(deleted)} records for {date_str}")
            total_deleted += len(deleted)
        else:
            print(f"ℹ️ No records found for {date_str}")
    except Exception as e:
        print(f"⚠️ Error deleting for {date_str}: {e}")

    # Go to the next day
    current_date += timedelta(days=1)

    # Optional: small delay to avoid server overload
    time.sleep(0.5)

print(f"\n🎉 Finished. Total deleted: {total_deleted} records.")

ℹ️ No records found for 2023-05-30
ℹ️ No records found for 2023-05-31
✅ Deleted 41217 records for 2023-06-01
✅ Deleted 41217 records for 2023-06-02
✅ Deleted 41217 records for 2023-06-03
✅ Deleted 41217 records for 2023-06-04
✅ Deleted 41217 records for 2023-06-05
✅ Deleted 41217 records for 2023-06-06
✅ Deleted 41217 records for 2023-06-07
✅ Deleted 41217 records for 2023-06-08
✅ Deleted 41217 records for 2023-06-09
✅ Deleted 41217 records for 2023-06-10
✅ Deleted 41217 records for 2023-06-11
✅ Deleted 41217 records for 2023-06-12
✅ Deleted 41217 records for 2023-06-13
✅ Deleted 41217 records for 2023-06-14
✅ Deleted 41217 records for 2023-06-15
✅ Deleted 41217 records for 2023-06-16
✅ Deleted 41217 records for 2023-06-17
✅ Deleted 41217 records for 2023-06-18
✅ Deleted 41217 records for 2023-06-19
✅ Deleted 41217 records for 2023-06-20
✅ Deleted 41217 records for 2023-06-21
✅ Deleted 41217 records for 2023-06-22
✅ Deleted 41217 records for 2023-06-23
✅ Deleted 41217 records for 2023-

Finally we will push the geodatabase as an update. Can take a long time if it is large.

In [9]:
from arcgis.gis import GIS
gis = GIS("home")
from arcgis.features import FeatureLayerCollection

gdb_zip_path = "C:\ArcGIS\data_upload\historical\output_07_28_2025.zip"

# ======= Push update? =========
item_id = "e7c7151adfd549239c31f9cc2363831c"
item = gis.content.get(item_id)
flc = FeatureLayerCollection.fromitem(item)
related_table_combined = flc.tables[0]

source_item = gis.content.add({
    "title": "output_gdb",
    "type": "File Geodatabase"
}, data=gdb_zip_path)

result = related_table_combined.append(
    item_id=source_item.id,
    upload_format='filegdb',
    source_table_name='merged_output',
    upsert=False
)
print("Append result:", result)

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\Lib\site-packages\IPython\core\interactiveshell.py:3550: DeprecatedWarning: add is deprecated as of 2.3.0 and has been removed in 3.0.0. Use `Folder.add()` instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'msstate.maps.arcgis.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Append result: True


If you try the code cell above more than once you will get an error that whatever item already exsists. Copy the item ID below to delete it, then run the above code cell again.

In [8]:
# If you try to upload the table and it fails and you try again you will have to delete it before you can try again.
duplicate_item = gis.content.get("b9ade587be654b869479a0e21beec482")
duplicate_item.delete()


True